In [ ]:
###### NO standardddddizeeee

## DeepNeuralNet

In [34]:
from neuralnet.second_version import *

In [35]:
import numpy as np
import matplotlib.pyplot as plt

#### 1. Import Data

##### Linear

In [36]:
# from sklearn.datasets import load_boston
# import copy
# X,y = load_boston(return_X_y = True)
# print('shapes of X, y')
# print(X.shape,y.shape)

##### Binary

In [37]:
# from sklearn.datasets import load_breast_cancer
# import copy
# X,y = load_breast_cancer(return_X_y = True)
# print('shapes of X, y')
# print(X.shape,y.shape)

##### Multinomial

In [41]:
# from sklearn.datasets import load_digits
# import copy
# X,y = load_digits(return_X_y = True)
# print('shapes of X, y')
# print(X.shape,y.shape)

In [42]:
import scipy.io
mat = scipy.io.loadmat('mnist-original.mat')

In [43]:
X = mat['data'].T
y = mat['label'].T
# print(y.shape)
print('shapes of X, y')
print(X.shape,y.shape)

shapes of X, y
(70000, 784) (70000, 1)


#### 1.1 Split data

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)

#since our train function assumes y to be shape of (n, 1)
y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

# double check the shapes
assert X_train.shape[0] == y_train.shape[0]
# assert len(X_train)  == len(y_train)
assert X_test.shape[0] == y_test.shape[0]
# assert len(X_test) == len(y_test)

print('shapes of X_train, y_train')
print(X_train.shape,y_train.shape)
print('shapes of X_test, y_test')
print(X_test.shape,y_test.shape)



shapes of X_train, y_train
(49000, 784) (49000, 1)
shapes of X_test, y_test
(21000, 784) (21000, 1)


#### 2.Data Preprocessing

##### 2.1 Standardization/Normalization

In [21]:
##### NO

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##### 2.2 One Hot (if necessary)

In [45]:
from sklearn import preprocessing
onehot = preprocessing.OneHotEncoder()

#sklearn expects a 2D array thus we have to reshape to (-1, 1)
y_train_encode = onehot.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_encode = onehot.fit_transform(y_test.reshape(-1, 1)).toarray()

print(y_train_encode.shape, y_test_encode.shape)

(49000, 10) (21000, 10)


#### 3. Accuracy Functions

##### 3.1 Linear

In [46]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def calc_accuracy_linear(model, X_test, y_test):    
    #getting the accuracy score with testing data
    preds = model.forward(X_test)
    print("MSE: ", mean_squared_error(y_test, preds))
    print("r2 score: ", r2_score(y_test, preds))


##### 3.2 Classification

##### 3.2.1 binary

In [47]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def calc_accuracy_binary_classification(model, X_test, y_test):    
    #getting the accuracy score with testing data
    preds = model.forward(X_test)
    preds[preds>0.5] = 1
    preds[preds<0.5] = 0
    print("Accuracy: ", accuracy_score(y_test, preds))
    print("Classification Report: ", classification_report(y_test, preds))

##### 3.2.2 multinomial

In [48]:
#define a simple accuracy function
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def calc_accuracy_multinomial_classification(model, X_test, y_test):    
    #getting the accuracy score with testing data
    preds = model.forward(X_test)
    preds = np.argmax(preds, axis=1)
    print("Accuracy: ", accuracy_score(y_test, preds))
    print("Classification Report: ", classification_report(y_test, preds))

#### 4. Add more fancy stuffs

In [49]:
class Relu(Operation):
    def __init__(self) -> None:
        super().__init__()
    
    def _output(self, inference: bool) -> ndarray: 
        return np.maximum(0,self.input_)

    def _input_grad(self, output_grad: ndarray) -> ndarray:
        self.output[self.output <= 0] = 0
        self.output[self.output > 0] = 1
        return output_grad * self.output 

class LeakyRelu(Operation):
    def __init__(self) -> None:
        super().__init__()
    
    def _output(self, inference: bool) -> ndarray: 
        return np.maximum(0.2*self.input_,self.input_)

    def _input_grad(self, output_grad: ndarray) -> ndarray:
        dz = np.ones_like(self.output)
        dz[self.output<0] = 0.2
        return output_grad * dz


In [50]:
class BinaryCrossEntropy(Loss):
    def __init__(self, eps: float=1e-9):
        super().__init__()
        self.eps = eps

    def _output(self) -> float:

        # clipping the softmax output to prevent numeric instability
        #numpy.clip(a, a_min, a_max, out=None, **kwargs)
        #To prevent extremely large loss values that could lead to numeric instability, 
        #we’ll clip the output of the softmax function to be no less than 10–7 and no greater than 10^7
        self.prediction = np.clip(self.prediction, self.eps, 1 - self.eps)

        # actual loss computation
        binary_cross_entropy_loss = (
            -1.0 * self.target * np.log(self.prediction) - \
                (1.0 - self.target) * np.log(1 - self.prediction)
        )
        
        #return average loss
        return np.sum(binary_cross_entropy_loss) / self.prediction.shape[0]

    def _input_grad(self) -> ndarray:
        #return average grad
        return (self.prediction - self.target) / self.prediction.shape[0]


### Main Code: WATCH OUT FOR THE NUMBER OF NEURONS

##### Model: Linear

In [14]:
# model = NeuralNetwork(
#     layers=[Dense(neurons=13, 
#                   activation=Sigmoid(),
#                   weight_init="glorot",
#                   dropout=0.8),
#             Dense(neurons=13, 
#                   activation=Sigmoid(),
#                   weight_init="glorot",
#                   dropout=0.8),
#             Dense(neurons=1,  ##### Neurons = 1
#                   activation=Linear(),  ## END WITH LINEAR
#                   weight_init="glorot")],
#             loss = MeanSquaredError(), 
# seed=20200720)

In [15]:
# trainer = Trainer(model,SGD(lr=0.01) )
# #SGDMomentum(lr=0.2, momentum=0.9,final_lr=0.05, decay_type='exponential'))
# trainer.fit(X_train, y_train, X_test, y_test,
#             epochs = 100,
#             eval_every = 10,
#             seed=20200720,
#             batch_size=60)

# print('========= Accuracy ========')
# calc_accuracy_linear(model, X_test, y_test)

##### Model: Binary

In [16]:
# model = NeuralNetwork(
#     layers=[Dense(neurons=20, 
#                   activation=Tanh(),
#                   weight_init="glorot",
#                   dropout=0.8),
#             Dense(neurons=5, 
#                   activation=Tanh(),
#                   weight_init="glorot",
#                   dropout=0.8),
#             Dense(neurons=1,  ##### Neurons = 1
#                   activation=Sigmoid(),  ##### SIGMOID
#                   weight_init="glorot")],
#             loss = MeanSquaredError(), 
# seed=20200720)

In [17]:
# model = NeuralNetwork(
#     layers=[Dense(neurons=20, 
#                   activation=Sigmoid(),
#                   weight_init="glorot",
#                   dropout=0.8),
#             Dense(neurons=5, 
#                   activation=Sigmoid(),
#                   weight_init="glorot",
#                   dropout=0.8),
#             Dense(neurons=1, 
#                   activation=Sigmoid(), ##### SIGMOID
#                   weight_init="glorot")],
#             loss = BinaryCrossEntropy(), 
# seed=20200720)

In [54]:
# trainer = Trainer(model,SGDMomentum(lr=0.2, momentum=0.9,
#                                     final_lr=0.05, decay_type='exponential'))
# trainer.fit(X_train, y_train, X_test, y_test,
#             epochs = 50,
#             eval_every = 10,
#             seed=20200720,
#             batch_size=50)

# print('========= Accuracy ========')
# calc_accuracy_binary_classification(model, X_test, y_test)

##### Model: Multinomial

In [52]:
model = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20200720)

In [53]:
trainer = Trainer(model, SGDMomentum(lr=0.2, momentum=0.9,
                                    final_lr=0.05, decay_type='exponential'))
trainer.fit(X_train, y_train_encode, X_test, y_test_encode,
            epochs = 50,
            eval_every = 10,
            seed=20200720,
            batch_size=60)

print('========= Accuracy ========')
calc_accuracy_multinomial_classification(model, X_test, y_test)

Validation loss after 10 epochs is 3.697
Validation loss after 20 epochs is 3.315
Loss increased after epoch 30, final loss was 3.315, using the model from epoch 20
========= Accuracy ========
Accuracy:  0.10171428571428572
Classification Report:                precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2013
         1.0       0.00      0.00      0.00      2320
         2.0       0.10      0.41      0.16      2140
         3.0       0.10      0.51      0.17      2098
         4.0       0.07      0.00      0.00      2047
         5.0       0.00      0.00      0.00      1971
         6.0       0.09      0.01      0.02      2092
         7.0       0.00      0.00      0.00      2206
         8.0       0.08      0.00      0.00      2052
         9.0       0.10      0.08      0.09      2061

    accuracy                           0.10     21000
   macro avg       0.05      0.10      0.04     21000
weighted avg       0.05      0.10      0.04     

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
